# Building first level models using _nipype_ and _SPM12_

## Base functionality for _megameta_ project

-------
#### History

* 3/29/19 mbod - split preprocessing (unzip, resample, smooth) from modeling steps
* 3/28/19 mbod - update pipeline to include resampling to template & SPM path reference
* 3/23/19 mbod - include contrast definition in the config JSON file
* 3/9/19 mbod - updates from testing template with `darpa1`
* 2/27/19 mbod  - modify example notebook to make base functionality notebook

-----

### Description

* Set up a nipype workflow to use SPM12 to make first level models for _megameta_ task data (preprocessed using `batch8` SPM8 scripts) in BIDS derivative format   


-------------------

### Template variables

* Specify the following values:
    1. project name - should be name of folder under `/data00/project/megameta`, e.g. `project1`
    2. filename for JSON model specification (should be inside `model_specification` folder), e.g. `p1_image_pmod_likeme.json`
    3. TR value in seconds
 



-------------------

### Setup

* import required modules and define parameters

In [1]:
import os  # system functions

# NIYPE FUNCTIONS
import nipype.interfaces.io as nio           # Data i/o
import nipype.interfaces.spm as spm          # spm
import nipype.interfaces.matlab as mlab      # how to run matlab
import nipype.interfaces.utility as util     # utility
import nipype.pipeline.engine as pe          # pypeline engine
import nipype.algorithms.modelgen as model   # model specification
from nipype.interfaces.base import Bunch
from nipype.algorithms.misc import Gunzip

from itertools import combinations

from nilearn import plotting, image
from nistats import thresholding


from IPython.display import Image


import scipy.io as sio
import numpy as np
import json
import pandas as pd
import glob

200616-12:15:31,19 nipype.utils INFO:
	 Running nipype version 1.4.2 (latest: 1.5.0)


/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: 

 | Starting with Nilearn 0.7.0, all Nistats functionality has been incorporated into Nilearn's stats & reporting modules.
 | Nistats package will no longer be updated or maintained.

  app.launch_new_instance()


In [2]:
#resolution='medium'
#PROJECT_NAME='alcohol'
#TASK_NAME='task'
#smoothing=[8]
#PATH_TO_SPM_FOLDER = '/data00/tools/spm12mega'

#### Matlab path


In [3]:
# Set the way matlab should be called
mlab.MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")
# If SPM is not in your MATLAB path you should add it here
mlab.MatlabCommand.set_default_paths(PATH_TO_SPM_FOLDER)

### Parameters

* These need to be reformatted to be consistent
* as data is not smoothed commented out the `fwhm_size` param - but data probably has a value

In [4]:
PROJECT_DIR = os.path.join('/data00/projects/megameta', PROJECT_NAME)
fmriprep_dir = os.path.join(PROJECT_DIR, 'derivatives','fmriprep')
batch8_dir=os.path.join(PROJECT_DIR, 'derivatives','batch8')

task_func_template = "{PID}_task-{TASK}_*bold*.nii.gz"

if os.path.exists(fmriprep_dir):
    SUBJ_DIR=fmriprep_dir
    all_subjects=[s for s in os.listdir(fmriprep_dir)]
if os.path.exists(batch8_dir):
    SUBJ_DIR=batch8_dir
    all_subjects=[s for s in os.listdir(batch8_dir)]

subject_list = [subj for subj in all_subjects
                   if glob.glob(os.path.join(SUBJ_DIR,subj,'func',
                                        task_func_template.format(PID=subj, TASK=TASK_NAME)))
                ]

output_dir = os.path.join(PROJECT_DIR,'derivatives', 'nipype','resampled_and_smoothed')        # name of output folder
working_dir = os.path.join(PROJECT_DIR, 'working', 
                           'nipype', 'workingdir_preproc_{}'.format(TASK_NAME.upper()))   # name of working directory



In [ ]:
# check to see if output and work directories exist

if not os.path.exists(output_dir):
    os.makedirs(output_dir) 

if not os.path.exists(working_dir):
    os.makedirs(working_dir)

In [ ]:

try:
    subject_list = [ s for s in subject_list if s not in exclude_subjects ]
    print('\n\nApplied subject inclusion list:\n\t',' '.join(exclude_subjects))
except:
    print('\n\nNo subject exclusions applied')

try:
    subject_list = [ s for s in subject_list if s in include_subjects ]
    print('\n\nApplied subject inclusion list:\n\t',' '.join(include_subjects))
except:
    print('\n\nNo subject inclusions applied')

    
print('\n\nSUBJECT LIST IS:\n\t', ' '.join(subject_list))

## Set up nodes for file handling and subject selection

### `infosource` node

* iterate over list of subject ids and generate subject ids and produce list of contrasts for subsequent nodes

In [5]:
# Infosource - a function free node to iterate over the list of subject names
infosource = pe.Node(util.IdentityInterface(fields=['subject_id', 'resolution']),
                  name="infosource")

ref_image_dict = {'low': '/data00/projects/megameta/templates/reference_low_wad.nii',
                  'medium': '/data00/projects/megameta/templates/reference_medium_wad.nii',
                  'high': '/data00/projects/megameta/templates/reference_high_wad.nii'}
             
if resolution:
    ref_images = [v for k,v in ref_image_dict.items() if k in resolution]
else:
    ref_images = ref_image.dict.values()
    
infosource.iterables = [('subject_id', subject_list),
                       ('resolution', ref_images)]



### `selectfiles` node

* match template to find source files (functional) for use in subsequent parts of pipeline

In [6]:
# SelectFiles - to grab the data (alternativ to DataGrabber)

if os.path.exists(fmriprep_dir):
    bd='/data00/projects/megameta/{}/derivatives/fmriprep'
    ## TODO: here need to figure out how to incorporate the run number and task name in call
    templates = {'func': '{subject_id}/func/{subject_id}_task-'+TASK_NAME+'_run-0*_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'}       
if os.path.exists(batch8_dir):
    bd='/data00/projects/megameta/{}/derivatives/batch8'
    ## TODO: here need to figure out how to incorporate the run number and task name in call
    templates = {'func': '{subject_id}/func/{subject_id}_task-'+TASK_NAME+'_run-0*_space-MNI152-T1-1mm_desc-preproc_bold.nii.gz'}    

selectfiles = pe.Node(nio.SelectFiles(templates,
                               base_directory=bd.format(PROJECT_NAME)),
                      working_dir=working_dir,
                   name="selectfiles")

## Unzip and smoothing steps

* BIDS derivatives folders contain unsmoothed functional NIFTI files in zipped (.nii.gz) format
* This subflow adds three nodes:
    1. gunzip
    2. resample
    3. smooth

#### Specify unzip node

* transform `.nii.gz` to `.nii`

In [ ]:
gunzip = pe.MapNode(Gunzip(),name="gunzip", iterfield=['in_file'])

#### Specify smoothing node

In [ ]:
smooth = pe.Node(interface=spm.Smooth(), name="smooth")
fwhmlist = [4,6,8]

if smoothing:
    fwhmlist = smoothing

smooth.iterables = ('fwhm', fwhmlist)

#### Specify resampling node

In [ ]:
resample = pe.MapNode(interface=spm.utils.Reslice(), 
                      name='resample',
                     iterfield=['in_file'])

In [ ]:
unzip_resample_and_smooth = pe.Workflow(name='unzip_resample_and_smooth')

unzip_resample_and_smooth.base_dir = os.path.join(SUBJ_DIR, working_dir)

unzip_resample_and_smooth.connect(
    [
        (gunzip, resample, [('out_file', 'in_file')]),
        (resample, smooth, [('out_file', 'in_files')])
    ]
)

### Specify preprocessing steps datasink node

* copy files to keep from various working folders to output folder for model for subject

In [ ]:
# Datasink - creates output folder for important outputs
pp_datasink = pe.Node(nio.DataSink(base_directory=SUBJ_DIR,
                         parameterization=True, 
                         #container=output_dir      
                               ),
                name="pp_datasink")

pp_datasink.inputs.base_directory = output_dir

# Use the following DataSink output substitutions
substitutions = []
subjFolders = [('_resolution_.*reference_(low|medium|high)_wad.nii_subject_id_%s/_fwhm_%s' % (sub,f), '\\1/fwhm_%s' % (f))
               for f in fwhmlist
               for sub in subject_list]
substitutions.extend(subjFolders)
pp_datasink.inputs.regexp_substitutions = substitutions

---------

## Set up workflow for preprocessing

In [ ]:
preprocess = pe.Workflow(name='preprocessing_steps')
preprocess.base_dir = os.path.join(SUBJ_DIR, working_dir)

preprocess.connect(
                    [ (infosource, selectfiles, [('subject_id', 'subject_id')]),
                      (infosource, unzip_resample_and_smooth, [('resolution', 
                                                                 'resample.space_defining')]),
                      (selectfiles, unzip_resample_and_smooth, [('func','gunzip.in_file')]),
                      (infosource, pp_datasink, [('subject_id','container')]),
                      (unzip_resample_and_smooth, pp_datasink, [('smooth.smoothed_files','@nii')])
                    ]
)
